# Cell motility study

In this notebook we will go through how we can use PhysiCOOL to test the effect of the cell motility parameters on the cells' trajectories and migration patterns. We will start by creating a simple black box model to run the simulation through Python and then adding other components to extend its functionalities.

## Creating the initial cell positions .csv file

We want to study the effect of migration parameters (namely the speed and migration_bias). Our PhysiCell model considers cells that are attracted to oxygen and there is an oxygen gradient from y_max to y_min. Thus, we will place the cells close to the y_min wall to then evaluate how much they migrated towards the opposite wall

In [2]:
import numpy as np

# Creates an array with the cell coordinates and saves it to a csv file.
# Cells will be placed in a line (y=-450), in the same z-plane (z=0).
cell_definition = 0
z_plane = 0
y_coordinate = -450
x_min = -450
x_max = 450
x_spacing = 100

coords = np.asarray([[x, y_coordinate, z_plane, cell_definition] 
                     for x in np.arange(x_min, x_max+1, x_spacing)])
                     
np.savetxt("config/cells.csv", coords, delimiter=",")

## Setting up the environment

We can use PhysiCOOL to compile our project with a Python command and create a black box that can be used to run the PhysiCell model (we must be in the same directory as the simulation files).

In [3]:
from physicool import optimization as opt

# Compiles the project and creates a black box object for it
opt.compile_project()    
black_box = opt.PhysiCellBlackBox()

g++ -march=native  -O3 -fomit-frame-pointer -mfpmath=both -fopenmp -m64 -std=c++11  -c ./BioFVM/BioFVM_vector.cpp 
g++ -march=native  -O3 -fomit-frame-pointer -mfpmath=both -fopenmp -m64 -std=c++11  -c ./BioFVM/BioFVM_mesh.cpp 
g++ -march=native  -O3 -fomit-frame-pointer -mfpmath=both -fopenmp -m64 -std=c++11  -c ./BioFVM/BioFVM_microenvironment.cpp 
g++ -march=native  -O3 -fomit-frame-pointer -mfpmath=both -fopenmp -m64 -std=c++11  -c ./BioFVM/BioFVM_solvers.cpp 
g++ -march=native  -O3 -fomit-frame-pointer -mfpmath=both -fopenmp -m64 -std=c++11  -c ./BioFVM/BioFVM_matlab.cpp
g++ -march=native  -O3 -fomit-frame-pointer -mfpmath=both -fopenmp -m64 -std=c++11  -c ./BioFVM/BioFVM_utilities.cpp 
g++ -march=native  -O3 -fomit-frame-pointer -mfpmath=both -fopenmp -m64 -std=c++11  -c ./BioFVM/BioFVM_basic_agent.cpp 
g++ -march=native  -O3 -fomit-frame-pointer -mfpmath=both -fopenmp -m64 -std=c++11  -c ./BioFVM/BioFVM_MultiCellDS.cpp
g++ -march=native  -O3 -fomit-frame-pointer -mfpmath=both -f

./modules/PhysiCell_geometry.cpp: In function ‘bool PhysiCell::load_cells_from_pugixml(pugi::xml_node)’:
./modules/PhysiCell_geometry.cpp:304:9: warning: ignoring return value of ‘int system(const char*)’, declared with attribute warn_unused_result [-Wunused-result]
  304 |   system("sleep 1");
      |   ~~~~~~^~~~~~~~~~~


g++ -march=native  -O3 -fomit-frame-pointer -mfpmath=both -fopenmp -m64 -std=c++11  -c ./custom_modules/custom.cpp
g++ -march=native  -O3 -fomit-frame-pointer -mfpmath=both -fopenmp -m64 -std=c++11  -o project BioFVM_vector.o BioFVM_mesh.o BioFVM_microenvironment.o BioFVM_solvers.o BioFVM_matlab.o BioFVM_utilities.o BioFVM_basic_agent.o BioFVM_MultiCellDS.o BioFVM_agent_container.o   pugixml.o PhysiCell_phenotype.o PhysiCell_cell_container.o PhysiCell_standard_models.o PhysiCell_cell.o PhysiCell_custom.o PhysiCell_utilities.o PhysiCell_constants.o PhysiCell_basic_signaling.o PhysiCell_signal_behavior.o  PhysiCell_SVG.o PhysiCell_pathology.o PhysiCell_MultiCellDS.o PhysiCell_various_outputs.o PhysiCell_pugixml.o PhysiCell_settings.o PhysiCell_geometry.o custom.o main.cpp 


main.cpp: In function ‘int main(int, char**)’:
main.cpp:106:8: warning: ignoring return value of ‘int system(const char*)’, declared with attribute warn_unused_result [-Wunused-result]
  106 |  system( copy_command );
      |  ~~~~~~^~~~~~~~~~~~~~~~


make name
make[1]: Entering directory '/workspace/abm-worskshop'

Executable name is project

make[1]: Leaving directory '/workspace/abm-worskshop'


## Running the model

Running a model with the black box is quite easy: we just have to call the run() method. Furthermore, we can choose the number of replicates to be run and whether the output data should be saved (it will be stored in a new folder called temp).


In [4]:
opt.clean_tmp_files()
black_box.run(number_of_replicates=1, keep_files=True)

## Plotting the results

PhysiCOOL offers functions to extract and plot the cell trajectories both in 3D and 2D.

In [5]:
%matplotlib notebook
from physicool.processing import get_cell_trajectories
from physicool.plotting import plot_trajectories_3d, plot_trajectories_2d

trajectories = get_cell_trajectories("temp")
plot_trajectories_3d(trajectories)

ValueError: No objects to concatenate

## Adding a params updater to the black box

We can add a ParamsUpdater instance to our black box to update the XML file with new parameter values before the simulation is run.


In [6]:
from physicool.updaters import CellUpdater, update_motility_values

# Define the updater we want to use (change motility data)
new_values = {"speed": 5.0, "migration_bias": 1.0}
updater = CellUpdater(updater_function=update_motility_values,
                     config_path="config/PhysiCell_settings.xml", cell_definition_name="default")

# Assign the updater to the black box
black_box.updater = updater

# Run the model with the new values
opt.clean_tmp_files()
black_box.run(params=new_values)

In [ ]:
# Plot the trajectories
trajectories = get_cell_trajectories("temp")
plot_trajectories_3d(trajectories)

## Adding a processing routine to extract the final positions

We can also add a OutputProcessor instance to extract and quantify data from our output files. In this case, we are interested in getting the y coordinates for the last time point, to assess how close the cells are at the end of the simulation to the y_max wall (oxygen source).

In [7]:
from physicool.processing import get_final_y_position

opt.clean_tmp_files()
black_box.processor = get_final_y_position
final_positions = black_box.run(new_values)

print("\n\n Cell final positions:")
print(final_positions)

ValueError: The passed time point does not match any file.